# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [25]:
# Instalando pacote selenium
!pip install selenium

In [26]:
# Importando pacotes
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [27]:
# Pegando a cotação do dólar
# Abrindo navegador
navegador = webdriver.Chrome()
# Entrando no google
navegador.get("https://www.google.com.br/")
# Pesquisando cotação dólar no google
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao dolar")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# Pegando a cotação
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)
# Fechando navegador
navegador.quit()

5.2081


In [28]:
# Pegando a cotação do euro
# Abrindo navegador
navegador = webdriver.Chrome()
# Entrando no google
navegador.get("https://www.google.com.br/")
# Pesquisando cotação dólar no google
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao euro")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# Pegando a cotação
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)
# Fechando navegador
navegador.quit()

5.289476562


In [29]:
# Pegando a cotação do ouro
# Abrindo navegador
navegador = webdriver.Chrome()
# Entrando no site da cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
# Pegando a cotação
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".") 
print(cotacao_ouro)
# Fechando navegador
navegador.quit()

293.47


### Agora vamos atualiza a nossa base de preços com as novas cotações

In [30]:
# Importando a base de dados
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [31]:
# Recalculando os preços da cotação
tabela.loc[tabela["Moeda"]=="Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro", "Cotação"] = float(cotacao_ouro)

# Recalculando o preço de compra
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Recalculando o preço de compra
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.208100,5208.047919,1.40,7291.267087
1,Carro Renault,4500.00,Euro,5.289477,23802.644529,2.00,47605.289058
2,Notebook Dell,899.99,Dólar,5.208100,4687.237919,1.70,7968.304462
3,IPhone,799.00,Dólar,5.208100,4161.271900,1.70,7074.162230
4,Carro Fiat,3000.00,Euro,5.289477,15868.429686,1.90,30150.016403
5,Celular Xiaomi,480.48,Dólar,5.208100,2502.387888,2.00,5004.775776
6,Joia 20g,20.00,Ouro,293.470000,5869.400000,1.15,6749.810000


### Agora vamos exportar a nova base de preços atualizada

In [32]:
# Exportando a base de dados
tabela.to_excel("ProdutosNovo.xlsx", index=False)